In [ ]:
import os
import sys

from google.colab import drive
drive.mount('/content/drive/')

os.chdir("/content/drive/MyDrive/Colab Notebooks/CS535")
print(f"This is my current working directory --> {os.getcwd()}")
import pandas as pd
import numpy as np
import io
import tarfile
import csv



# global variables
SEQ_LENGTH=20*60*100
BATCH_SIZE=15

Mounted at /content/drive/
This is my current working directory --> /content/drive/MyDrive/Colab Notebooks/CS535


In [ ]:
train_labels = pd.read_csv("AVEC_Challenge_Packed/labels/train_split.csv")
test_labels = pd.read_csv("AVEC_Challenge_Packed/labels/test_split.csv")
dev_labels = pd.read_csv("AVEC_Challenge_Packed/labels/dev_split.csv")

In [ ]:
def convert(df, df_name):
    labels = []
    for i in range(len(df)):
        temp = {}
        temp["ID"] = df.loc[i, "Participant_ID"]
        temp["Gender"] = df.loc[i, "Gender"]
        temp["Label"] = df.loc[i, "PHQ_Binary"]
        temp["Score"] = df.loc[i, "PHQ_Score"]
        temp["type"] = df_name
        labels.append(temp)
    return labels

In [ ]:
def get_combined_text(labels_df):
  labels = []
  #+ convert(dev_labels, "dev") + convert(test_labels, "test")
  labels = labels + convert(labels_df, "train") 
  labels_df = pd.DataFrame(labels, index = None)
  labels_df = labels_df.sort_values("ID")
  labels_df = labels_df.reset_index(drop = True)
  # labels_df = labels_df.drop(columns = ["index"])
  combined_transcripts = pd.DataFrame()
  for i in range(len(labels_df)):
      file = labels_df.loc[i, "ID"]
      file_name = "Transcripts/" + str(file) + "_transcript.csv"
      #try:
      text_df = pd.read_csv(file_name)
      #print("it worked")
      start_times = []
      texts = []
      end_times = []
      for i in range(len(text_df)):
        start_times.append(text_df.loc[ i,"Start_Time,End_Time,Text,Confidence"].split(",")[0])
        texts.append(text_df.loc[i,"Start_Time,End_Time,Text,Confidence"].split(",")[2])
        end_times.append(text_df.loc[i,"Start_Time,End_Time,Text,Confidence"].split(",")[1])
      text_df['Start_Time'] = start_times
      text_df['Text'] = texts
      text_df['End_Time'] = end_times 
      combined_transcripts = pd.concat([combined_transcripts, text_df], ignore_index=True, axis=0)
  return combined_transcripts
  # except:
    #     print(file)

In [ ]:
def compress_data(combined_trans,audio_df,name):
  compressed_avg_audio = pd.DataFrame()
  for i in range(len(combined_trans)):
    print(f'{i}/{len(combined_trans)}')
    start_time = combined_trans.loc[i, "Start_Time"]
    patient_id = combined_trans.loc[i, "patient_id"]
    #print(start_time)
    end_time = combined_trans.loc[i, "End_Time"]
    # print(end_time)
    # print(patient_id)
    temp_df = audio_df.loc[audio_df['patient_id'] == patient_id]
    temp_df = temp_df.loc[(temp_df['frameTime'] > float(start_time)) & (temp_df['frameTime'] < float(end_time))]
    temp_df = temp_df.mean().to_frame().T.iloc[:,2:]
    compressed_avg_audio = pd.concat([compressed_avg_audio, temp_df], ignore_index=True, axis=0)
  print("Done compressing")
  print("unique count:",len(compressed_avg_audio['patient_id'].unique()))
  compressed_avg_audio = compressed_avg_audio.rename(columns={"req": "reg"})
  df = pd.concat([combined_trans,compressed_avg_audio],ignore_index=False, axis=1)
  # df = df.drop("frameTime", axis=1)
  df = df.dropna(axis=0)
  df.to_pickle(name)
  return compressed_avg_audio

In [ ]:
3 < 1.00

False

In [ ]:
def compress_data_vgg16(combined_trans,audio_df,name):
  compressed_avg_audio = pd.DataFrame()
  for i in range(len(combined_trans)):
    print(f'{i}/{len(combined_trans)}')
    start_time = combined_trans.loc[i, "Start_Time"]
    patient_id = combined_trans.loc[i, "patient_id"]
    #print(start_time)
    end_time = combined_trans.loc[i, "End_Time"]
    # print(end_time)
    # print(patient_id)
    temp_df = audio_df.loc[audio_df['patient_id'] == patient_id]
    temp_df = temp_df.loc[(temp_df['timeStamp'] > float(start_time)) & (temp_df['timeStamp'] < float(end_time))]
    temp_df = temp_df.mean().to_frame().T.iloc[:,1:]
    compressed_avg_audio = pd.concat([compressed_avg_audio, temp_df], ignore_index=True, axis=0)
  print("Done compressing")
  print("unique count:",len(compressed_avg_audio['patient_id'].unique()))
  #compressed_avg_audio = compressed_avg_audio.rename(columns={"req": "reg"})
  df = pd.concat([combined_trans,compressed_avg_audio],ignore_index=False, axis=1)
  # df = df.drop("frameTime", axis=1)
  df = df.dropna(axis=0)
  df.to_pickle(name)
  return compressed_avg_audio

In [ ]:
def run(role,data):
  if role == 'train' and data == 'egemaps':
    text_train = get_combined_text(train_labels)
    egemaps_combined= pd.read_pickle("Fusion/labeled_egemaps_combined.pkl")
    egemaps_train = egemaps_combined.loc[egemaps_combined['patient_id'].isin(train_labels['Participant_ID'])]
    del egemaps_combined
    df = compress_data(text_train,egemaps_train,'compressed/updated_compressed_egemaps_train.pkl')
    return text_train, df

  if role == 'dev' and data == 'egemaps':
    text_dev = get_combined_text(dev_labels)
    egemaps_combined= pd.read_pickle("Fusion/labeled_egemaps_combined.pkl")
    egemaps_dev = egemaps_combined.loc[egemaps_combined['patient_id'].isin(dev_labels['Participant_ID'])]
    del egemaps_combined
    df = compress_data(text_dev,egemaps_dev,'compressed/updated_compressed_egemaps_dev.pkl')
    return text_dev, df

  if role == 'train' and data == 'mfcc':
    text_train = get_combined_text(train_labels)
    mfcc_combined= pd.read_pickle("Fusion/labeled_mfcc_combined.pkl")
    mfcc_train = mfcc_combined.loc[mfcc_combined['patient_id'].isin(train_labels['Participant_ID'])]
    del mfcc_combined
    df = compress_data(text_train,mfcc_train,'compressed/updated_compressed_mfcc_train.pkl')
    return text_train, df

  if role == 'dev' and data == 'mfcc':
    text_dev = get_combined_text(dev_labels)
    mfcc_combined= pd.read_pickle("Fusion/labeled_mfcc_combined.pkl")
    mfcc_dev = mfcc_combined.loc[mfcc_combined['patient_id'].isin(dev_labels['Participant_ID'])]
    del mfcc_combined
    df = compress_data(text_dev,mfcc_dev,'compressed/updated_compressed_mfcc_dev.pkl')
    return text_dev, df

  if role == 'test' and data == 'mfcc':
    text_test = get_combined_text(test_labels)
    mfcc_combined= pd.read_csv("Fusion/labeled_mfcc_combined_test.csv")
    mfcc_test = mfcc_combined.loc[mfcc_combined['patient_id'].isin(test_labels['Participant_ID'])]
    del mfcc_combined
    df = compress_data(text_test,mfcc_test,'compressed/updated_compressed_mfcc_test.pkl')
    return text_test, df
  if role == 'test' and data == 'egemaps':
    text_test = get_combined_text(test_labels)
    egemaps_combined= pd.read_csv("Fusion/labeled_egemaps_combined_test.csv")
    egemaps_test = egemaps_combined.loc[egemaps_combined['patient_id'].isin(test_labels['Participant_ID'])]
    del egemaps_combined
    df = compress_data(text_test,egemaps_test,'compressed/updated_compressed_egemaps_test.pkl')
    return text_test, df
  if role == 'test' and data == 'vgg16':
    text_test = get_combined_text(test_labels)
    vgg16_combined= pd.read_csv("Fusion/labeled_vgg16_combined.csv")
    vgg16_test = vgg16_combined.loc[vgg16_combined['patient_id'].isin(test_labels['Participant_ID'])]
    del vgg16_combined
    df = compress_data_vgg16(text_test,vgg16_test,'compressed/updated_compressed_vgg16_test.pkl')
    return text_test, df 
  if role == 'train' and data == 'vgg16':
    text_train = get_combined_text(train_labels)
    vgg16_combined= pd.read_csv("Fusion/labeled_vgg16_combined.csv")
    vgg16_train = vgg16_combined.loc[vgg16_combined['patient_id'].isin(train_labels['Participant_ID'])]
    del vgg16_combined
    df = compress_data_vgg16(text_train,vgg16_train,'compressed/updated_compressed_vgg16_train.pkl')
    return text_train, df 
  if role == 'dev' and data == 'vgg16':
    text_dev = get_combined_text(dev_labels)
    vgg16_combined= pd.read_csv("Fusion/labeled_vgg16_combined.csv")
    vgg16_dev = vgg16_combined.loc[vgg16_combined['patient_id'].isin(dev_labels['Participant_ID'])]
    del vgg16_combined
    df = compress_data_vgg16(text_dev,vgg16_dev,'compressed/updated_compressed_vgg16_dev.pkl')
    return text_dev, df 

In [ ]:

'updated_compressed_egemaps_dev.pkl'
'updated_compressed_egemaps_test.pkl'
'updated_compressed_egemaps_dev.pkl'
---
'updated_compressed_mfcc_train.pkl'
'updated_compressed_mfcc_dev.pkl'
'updated_compressed_mfcc_test.pkl'

In [ ]:
text,audio = run('dev','egemaps')

In [ ]:
text,audio = run('train','egemaps')

In [ ]:
text,audio = run('test','egemaps')

In [ ]:
text,audio = run('train','mfcc')

In [ ]:
text,audio = run('test','egemaps')

In [ ]:
text,audio = run('test','vgg16')

Streaming output truncated to the last 5000 lines.
573/5571
574/5571
575/5571
576/5571
577/5571
578/5571
579/5571
580/5571
581/5571
582/5571
583/5571
584/5571
585/5571
586/5571
587/5571
588/5571
589/5571
590/5571
591/5571
592/5571
593/5571
594/5571
595/5571
596/5571
597/5571
598/5571
599/5571
600/5571
601/5571
602/5571
603/5571
604/5571
605/5571
606/5571
607/5571
608/5571
609/5571
610/5571
611/5571
612/5571
613/5571
614/5571
615/5571
616/5571
617/5571
618/5571
619/5571
620/5571
621/5571
622/5571
623/5571
624/5571
625/5571
626/5571
627/5571
628/5571
629/5571
630/5571
631/5571
632/5571
633/5571
634/5571
635/5571
636/5571
637/5571
638/5571
639/5571
640/5571
641/5571
642/5571
643/5571
644/5571
645/5571
646/5571
647/5571
648/5571
649/5571
650/5571
651/5571
652/5571
653/5571
654/5571
655/5571
656/5571
657/5571
658/5571
659/5571
660/5571
661/5571
662/5571
663/5571
664/5571
665/5571
666/5571
667/5571
668/5571
669/5571
670/5571
671/5571
672/5571
673/5571
674/5571
675/5571
676/5571
677/5571
678/